# Face detection with Retinaface

### Reference

[1] [[Paper](https://arxiv.org/abs/1905.00641)] [[Github](https://github.com/biubug6/Pytorch_Retinaface.git)] Deng, Jiankang and Guo, Jia and Yuxiang, Zhou and Jinke Yu and Irene Kotsia and Zafeiriou, Stefanos, "RetinaFace: Single-stage Dense Face Localisation in the Wild", arxiv, 2019

[2] [[Paper](https://openaccess.thecvf.com/content_cvpr_2016/html/Yang_WIDER_FACE_A_CVPR_2016_paper.html)] [[Dataset](http://shuoyang1213.me/WIDERFACE/WiderFace_Results.html)] Shuo Yang, Ping Luo, Chen Change Loy, and Xiaoou Tang, "WIDER FACE: A Face Detection Benchmark", Proceedings of the IEEE conference on computer vision and pattern recognition, 2016

### Settings
- Create virtual environment and install library
    - conda create -n retinaface python=3.8
    - conda activate retinaface
    - pip install torch torchvision
    - pip install tqdm PyYAML scipy mxnet opencv-python
    - pip install matplotlib python-time torchsummary

- Download the Retinaface & WIDERFACE dataset & annotations 
    - [Retinaface github & Annotations & Pretrained model](https://github.com/biubug6/Pytorch_Retinaface?tab=readme-ov-file#data)
    - [WIDERFACE datset](http://shuoyang1213.me/WIDERFACE/WiderFace_Results.html)

- Place the files as follows:

    ```
    Pytorch_Retinaface-master                   # from Retinaface git
    └── dataset                                 # Dataset
        └── widerface
            └── WIDER_train
                └── images/
                └── train_label.txt
            └── WIDER_val/
                └── images/
                └── val_label.txt
    └── weights                                 # pretrained weights
        └── mobilenet0.25_Final.pth

    ```    

### Inference

In [1]:
# 라이브러리 삽입
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2
import torch
from data import cfg_mnet, cfg_re50
from models.retinaface import RetinaFace
from utils.nms.py_cpu_nms import py_cpu_nms
from layers.functions.prior_box import PriorBox
from utils.box_utils import decode

ModuleNotFoundError: No module named 'data'

In [ ]:
# 경로 설정
weight_path = "mobilenet0.25_Final.pth" #.pth
test_path =                             # test image

In [ ]:
# 하이퍼 파라미터 설정
cfg = cfg_mnet # mobile0.25 (cfg_mnet) or resnet50 (cfg_re50)
resize = 1
confidence_threshold = 0.02
top_k = 5000
nms_threshold = 0.4
keep_top_k = 750
vis_thres = 0.6

In [ ]:
# gpu 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# 모델 불러오기
model=RetinaFace(cfg, phase = 'test').to(device)
model.load_state_dict(torch.load(weight_path, map_location=device))
# 평가 모드로 전환하기
model.eval()
print("Model Loaded")

In [ ]:
# Retinaface Inference 함수
def retinaface_inf(test_img, model):
    img = np.float32(test_img)
    im_height, im_width, _ = img.shape

    scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
    img -= (104, 117, 123)
    img = img.transpose(2, 0, 1)
    img = torch.from_numpy(img).unsqueeze(0)
    img = img.to(device)
    scale = scale.to(device)

    tic = time.time()
    loc, conf, landms = model(img)  # forward pass

    priorbox = PriorBox(cfg, image_size=(im_height, im_width))
    priors = priorbox.forward()
    priors = priors.to(device)
    prior_data = priors.data
    boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
    boxes = boxes * scale / resize
    boxes = boxes.cpu().numpy()
    scores = conf.squeeze(0).data.cpu().numpy()[:, 1]

    # ignore low scores
    inds = np.where(scores > confidence_threshold)[0]
    boxes = boxes[inds]
    scores = scores[inds]

    # keep top-K before NMS
    order = scores.argsort()[::-1][:top_k]
    boxes = boxes[order]
    scores = scores[order]

    # do NMS
    dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
    keep = py_cpu_nms(dets, nms_threshold)
    dets = dets[keep, :]

    # keep top-K faster NMS
    dets = dets[:keep_top_k, :]

    fps_ = round(1/(time.time() - tic), 2)
    for b in dets:
        if b[4] < vis_thres:
            continue
        b = list(map(int, b))
        cv2.rectangle(test_img, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 4)
    cv2.putText(test_img, "retinaface", (410,70),cv2.FONT_HERSHEY_DUPLEX, 1.5,(255,0,0), thickness=3, lineType=cv2.LINE_AA)
    cv2.putText(test_img, "fps : "+str(fps_), (5,70),cv2.FONT_HERSHEY_DUPLEX, 1.5,(0,0,255), thickness=3, lineType=cv2.LINE_AA)
    return test_img

In [ ]:
# test image
test_img = cv2.imread(test_path)

retina_img = retinaface_inf(test_img, model)

plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
plt.show()